# 谢钢GPS原理与接收机设计 P61 例题3.1

导航电文如下, 计算$t_{gps} = 239050.7223 s$ 时的卫星空间位置 <br>
$ t_{oe} = 244800$ , <br>
$ \sqrt{a_s} = 5153.65531$ , <br>
$ e_s = 0.005912038265$ , <br>
$ i_0 = 0.9848407943$ , <br>
$ \Omega_0 = 1.038062244 $ , <br>
$ \omega = -1.717457876 $ , <br>
$ M_0 = -1.064739758 $ , <br>
$ \Delta n = 4.249105564 * 10^{-9} $ , <br>
$ \dot i = 7.422851197 * 10^{-51} $ , <br>
$ \dot \Omega = -8.151768125 * 10^{-9} $ , <br>
$ C_{uc} = 3.0541738045 * 10^{-7} $ , <br>
$ C_{us} = 2.237036824 * 10^{-6} $ , <br>
$ C_{rc} = 350.53125 $ , <br>
$ C_{rs} = 2.53125 $ , <br>
$ C_{ic} = -8.381903172 * 10^{-8} $ , <br>
$ C_{is} = 8.940696716 * 10^{-8} $ <br>


In [1]:
# 电文参数
t_oe = 244800
sr_as = 5153.65531  # square_root_as
e_s = 0.005912038265
i_0 = 0.9848407943
Omega_0 = 1.038062244
omega = -1.717457876
M_0 = -1.064739758
Delta_n = 4.249105564 * 10 ** (-9)
dot_i = 7.422851197 * 10 ** (-51)
dot_Omega = -8.151768125 * 10 ** (-9)
C_uc = 3.0541738045 * 10 ** (-7)
C_us = 2.237036824 * 10 ** (-6)
C_rc = 350.53125
C_rs = 2.53125
C_ic = -8.381903172 * 10 ** (-8)
C_is = 8.940696716 * 10 ** (-8)

t = 239050.7223
a_s = sr_as ** 2


# WGS-84的基本大地参数 <br>

基本椭球体的长半径 $a = 6378137.0 m$ <br>
基准椭球体的极扁率 $f = 1/298.257223563 $ <br>
地球自转角速度 $ \dot{\Omega}_e = 7.2921151467 * 10^{-5} rad/s $ <br>
地球引力与地球质量乘积 $GM = 3.986005 * 10^{14}  m^3/s^2 $ <br>
真空中的光速 $ c = 2.99792458 * 10^8 m/s $


In [2]:
GM = 3.986005 * 10 ** 14
dot_Omega_earth = 7.2921151467 * 10 ** (-5)

1. 计算规化时间  <br>
$ t_k = t - t_{oe}  $


In [3]:
t_k = t - t_oe
t_k

-5749.277700000006

2. 计算卫星平均角速度n <br>
$ n_0 = \sqrt{\frac{GM}{a_s^3}} = \sqrt{\frac{ \mu }{a_s^3}}, \mu = 3.986005 * 10^{14} \frac{m^3}{s^2} $ <br> 
$ n = n_0 + \Delta_n $ <br>


In [4]:
from math import sqrt

n_0 = sqrt(GM / a_s ** 3)
n = n_0 + Delta_n
n

0.00014585975041316255

3. 计算平近点角M_k <br>
$ M_k = M_0 + n * t_k $


In [5]:
M_k = M_0 + n * t_k
M_k

-1.903327968377962

归化到$[0, 2\pi]$


In [6]:
from math import pi


def norm_0_2pi(m):
    if m > 2 * pi:
        return norm_0_2pi(m - 2 * pi)
    elif m < 0:
        return norm_0_2pi(m + 2 * pi)
    else:
        return m


M_k = norm_0_2pi(M_k)
M_k

4.379857338801624

4. 计算偏近点角 <br>
$ M = E - e_s * sin(E) $ <br>
$ E_{k} = M + e_s * sin(E_{k - 1}) $


In [7]:
from math import sin

E_k = 0
E_k_1 = 0
for _ in range(100):
    E_k = M_k + e_s * sin(E_k_1)
    print(E_k)
    if abs(E_k - E_k_1) < 0.0001:
        break
    E_k_1 = E_k

4.379857338801624
4.374269168189134
4.374280040041773


5. 计算真近点角$v_k$

$ cos(\upsilon) = \frac {cos(E) - e_s} {1 - e_s cos(E)} $ <br>
$ sin(\upsilon) = \frac {\sqrt{1 - e_s^2} sin(E)} {1 - e_s cos(E)} $  <br>
$ tan(\upsilon) = \frac{sin(\upsilon)}{cos(\upsilon)} $  <br>
$ \upsilon = arctan(\frac{sin(\upsilon)}{cos(\upsilon)}) $


In [8]:
from math import sqrt, sin, cos, atan2


cos_v = (cos(E_k) - e_s) / (1 - e_s * cos(E_k))
sin_v = (sqrt(1 - e_s ** 2) * sin(E_k)) / (1 - e_s * cos(E_k))
tan_v = sin_v / cos_v
v_k = atan2(sin_v, cos_v)
v_k

-1.9144771581123803

归化到$(-\pi, \pi]$


In [9]:
def norm_pi_pi(v):
    if v > pi:
        return norm_pi_pi(v - 2 * pi)
    elif v < -pi:
        return norm_pi_pi(v + 2 * pi)
    else:
        return v


v_k = norm_pi_pi(v_k)
v_k

-1.9144771581123803

6. 计算卫星发射时刻的升交点角距 $ \Phi_k$ <br>
$ \Phi_k = v_k + \omega $


In [10]:
phi_k = v_k + omega
phi_k

-3.6319350341123804

7. 摄动校正项 <br>
$ \delta{u_k} = C_{us} sin(2 \Phi_k) + C_{uc} cos(2 \Phi_k)$ <br>
$ \delta{r_k} = C_{rs} sin(2 \Phi_k) + C_{rc} cos(2 \Phi_k)$ <br>
$ \delta{i_k} = C_{is} sin(2 \Phi_k) + C_{ic} cos(2 \Phi_k)$ <br>


In [11]:
sin_2_phi = sin(2 * phi_k)
cos_2_phi = cos(2 * phi_k)
delta_u_k = C_us * sin_2_phi + C_uc * cos_2_phi
delta_r_k = C_rs * sin_2_phi + C_rc * cos_2_phi
delta_i_k = C_is * sin_2_phi + C_ic * cos_2_phi
print(delta_u_k, delta_r_k, delta_i_k)

-1.6887553926791502e-06 192.95125796625567 -1.2092774832095e-07


8. 校正 升交点角距 $u_k$, 卫星矢径长度$r_k$, 轨道倾角$i_k$ <br>
$ u_k = \Phi_k + \delta u_k $ <br>
$ r_k = a_s(1-e_s cos(E_k)) + \delta r_k $ <br>
$ i_k = i_0 + \dot{i} t_k + \delta{i_k} $


In [12]:
u_k = phi_k + delta_u_k
r_k = a_s * (1 - e_s * cos(E_k)) + delta_r_k
i_k = i_0 + dot_i * t_k + delta_i_k

print(u_k, r_k, i_k)

-3.6319367228677732 26612441.67831102 0.9848406733722517


9. 计算卫星在轨道平面的位置$(x_k^’,y_k^’)$


In [13]:
x_k_tmp = r_k * cos(u_k)
y_k_tmp = r_k * sin(u_k)
z_k_tmp = 0
print(x_k_tmp, y_k_tmp)

-23476721.053770237 12532582.361387728


*Notes*
* 书中的x结果为 -23476720.79
* python结果为  -23476721.053770237
* matlab, windows计算器 计算结果一致, 为 -23476717.58207684, 相差在m级


10. 计算升交点赤经$\Omega_k$ <br>
$\Omega_k = \Omega_0 + (\dot{\Omega} - \dot{\Omega}_e) t_k - \dot{\Omega}_e t_{oe} $


In [14]:
Omega_k = Omega_0 + (dot_Omega - dot_Omega_earth) * t_k - dot_Omega_earth * t_oe
Omega_k

-16.39374481835536

In [15]:
Omega_k = norm_0_2pi(Omega_k)
Omega_k

2.4558111031834002

11. 计算WGS-84下ECEF的坐标 <br>
$ x_k = x_k^’ * cos(\Omega_k) - y_k^’cos(i_k)sin(\Omega_k) $<br>
$ y_k = x_k^’ * sin(\Omega_k) + y_k^’cos(i_k)cos(\Omega_k) $<br>
$ z_k = y_k^’sin(i_k)$<br>


In [16]:
x_k = x_k_tmp * cos(Omega_k) - y_k_tmp * cos(i_k) * sin(Omega_k)
y_k = x_k_tmp * sin(Omega_k) + y_k_tmp * cos(i_k) * cos(Omega_k)
z_k = y_k_tmp * sin(i_k)
print(x_k, y_k, z_k)

13780293.675619591 -20230949.077383496 10441947.027422614


12. 计算卫星速度 <br>
$ \dot{x_k} = (x_k^’ - y_k^’ \dot{\Omega_k}cos(\Omega_k) - (x_k^’\dot{\Omega_k} + y_k^’ cos(i_k) - y_k^’\dot{i_k}sin(i_k))sin(\Omega_k) $<br>
$ = -y_k\dot{\Omega_k} - (\dot{y_k^’} cos(i_k) - z_k\dot{i_k})sin(\Omega_k) + \dot{x}_k^’cos(\Omega_k) $ <br>
$ \dot{y_k} = (\dot{x}_k^’ - y_k^’ \dot{\Omega_k}cos(i_k))sin(\Omega_k) + (x_k^’\dot{\Omega_k} + \dot{_k^’} cos(i_k) - y_k^’\dot{i_k}sin(i_k)) cos(\Omega_k) $<br>
$ = x_k \dot{\Omega_k} + (\dot{y}_k^’cos(i_k) -z_k \dot{i_k})cos({\Omega_k}) + \dot{x}_k^’sin(\Omega_k) $ <br>
$ \dot{z_k} = \dot{y}_k sin(i_k) + y_k^’ \dot{i_k}cos(i_k)$<br>
其中，<br>
$ \dot{x}_k^’ = \dot{r}_k cos(u_k) - r_k \dot{u}_k sin(u_k)  $ <br>
$ \dot{y}_k^’ = \dot{r}_k sin(u_k) + r_k \dot{u}_k cos(u_k)  $ <br>
其中，<br>
$ \dot{u}_k = \dot{\Phi}_k + \delta\dot{u}_k $  <br>
$ \dot{r}_k = a_s e_s \dot{E}_k sin(E_k) + \delta\dot{r}_k $   <br>
$ \dot{i}_k = \dot{i} + \delta\dot{i}_k $ <br>
$ \dot{\Omega}_k = \dot{\Omega} - \dot{\Omega}_e $ <br>
其中，<br>
$ \delta\dot{u}_k = 2 \dot{\Phi}_k (C_{us} cos(2\Phi_k) - C_{uc} sin(2\Phi_k)) $ <br>
$ \delta\dot{r}_k = 2 \dot{\Phi}_k (C_{rs} cos(2\Phi_k) - C_{rc} sin(2\Phi_k)) $ <br>
$ \delta\dot{i}_k = 2 \dot{\Phi}_k (C_{is} cos(2\Phi_k) - C_{ic} sin(2\Phi_k)) $ <br>
其中，<br>
$ \delta\dot{\Phi}_k = \dot{v}_k $ <br>
$ \dot{v}_k = \frac {(1+e_scos(v_k))\dot{E}_ksin(E_k)} {(1 - e_s cos(E_k))sin{v_k}} = \frac{\sqrt{1-e_s^2} \dot{E}_k} {1 - e_s cos(E_k)} $ <br>
其中，<br>
$ \dot{E}_k = \frac{\dot{M}_k}{1 -e_s cos(E_k)}, \dot{M}_k = n $


下面再根据依赖关系，依次计算<br>
$ \dot{E}_k = \frac{\dot{M}_k}{1 -e_s cos(E_k)}, \dot{M}_k = n $


In [17]:
dot_E_k = n / (1 - e_s * cos(E_k))
dot_E_k

0.00014557427272553583

$ \dot{v}_k = \frac {(1+e_scos(v_k))\dot{E}_ksin(E_k)} {(1 - e_s cos(E_k))sin{v_k}} = \frac{\sqrt{1-e_s^2} \dot{E}_k} {1 - e_s cos(E_k)} $ <br>


In [18]:
dot_Phi_k = dot_v_k = sqrt(1 - e_s * e_s) * dot_E_k / (1 - e_s * cos(E_k))
dot_v_k

0.00014528681466356095

$ \delta\dot{u}_k = 2 \dot{\Phi}_k (C_{us} cos(2\Phi_k) - C_{uc} sin(2\Phi_k)) $ <br>
$ \delta\dot{r}_k = 2 \dot{\Phi}_k (C_{rs} cos(2\Phi_k) - C_{rc} sin(2\Phi_k)) $ <br>
$ \delta\dot{i}_k = 2 \dot{\Phi}_k (C_{is} cos(2\Phi_k) - C_{ic} sin(2\Phi_k)) $ <br>


In [19]:
dot_delta_u_k = 2 * dot_Phi_k * (C_us * cos(2 * phi_k) - C_uc * sin(2 * phi_k))
dot_delta_r_k = 2 * dot_Phi_k * (C_rs * cos(2 * phi_k) - C_rc * sin(2 * phi_k))
dot_delta_i_k = 2 * dot_Phi_k * (C_is * cos(2 * phi_k) - C_ic * sin(2 * phi_k))
print(dot_delta_u_k, dot_delta_r_k, dot_delta_i_k)

4.3544557394691074e-10 0.0850385341503285 -5.7802650261722235e-12


$ \dot{u}_k = \dot{\Phi}_k + \delta\dot{u}_k $  <br>
$ \dot{r}_k = a_s e_s \dot{E}_k sin(E_k) + \delta\dot{r}_k $   <br>
$ \dot{i}_k = \dot{i} + \delta\dot{i}_k $ <br>
$ \dot{\Omega}_k = \dot{\Omega} - \dot{\Omega}_e $ <br>


In [20]:
dot_u_k = dot_Phi_k + dot_delta_u_k
dot_r_k = a_s * e_s * dot_E_k * sin(E_k) + dot_delta_r_k
dot_i_k = dot_i + dot_delta_i_k
dot_Omega_k = dot_Omega - dot_Omega_earth

print(dot_u_k, dot_r_k, dot_i_k, dot_Omega_k)

0.0001452872501091349 -21.47953804395943 -5.7802650261722235e-12 -7.292930323512499e-05


$ \dot{x}_k^’ = \dot{r}_kcos(u_k) - r_k\dot{u}_k sin(u_k)  $ <br>
$ \dot{y}_k^’ = \dot{r}_ksin(u_k) + r_k\dot{u}_k cos(u_k)  $ <br>


In [21]:
dot_x_k_tmp = dot_r_k * cos(u_k) - r_k * dot_u_k * sin(u_k)
dot_y_k_tmp = dot_r_k * sin(u_k) + r_k * dot_u_k * cos(u_k)
print(dot_x_k_tmp, dot_y_k_tmp)

-1801.875805106754 -3420.9835903885864


$ \dot{x_k} = -y_k\dot{\Omega_k} - (\dot{y_k^’} cos(i_k) - z_k\dot{i_k})sin(\Omega_k) + \dot{x}_k^’cos(\Omega_k) $ <br>
$ \dot{y_k} =  x_k \dot{\Omega_k} + (\dot{y}_k^’cos(i_k) -z_k \dot{i_k})cos({\Omega_k}) + \dot{x}_k^’sin(\Omega_k) $ <br>
$ \dot{z_k} = \dot{y}_k^’ sin(i_k) + y_k^’ \dot{i_k}cos(i_k)$<br>


In [22]:
dot_x_k = (
    -y_k * dot_Omega_k
    - (dot_y_k_tmp * cos(i_k) - z_k * dot_i_k) * sin(Omega_k)
    + dot_x_k_tmp * cos(Omega_k)
)
dot_y_k = (
    x_k * dot_Omega_k
    + (dot_y_k_tmp * cos(i_k) - z_k * dot_i_k) * cos(Omega_k)
    + dot_x_k_tmp * sin(Omega_k)
)
dot_z_k = dot_y_k_tmp * sin(i_k) + y_k_tmp * dot_i_k * cos(i_k)

print(dot_x_k, dot_y_k, dot_z_k)

1117.1154766572486 -681.9735088321646 -2850.308811425085
